In [6]:
from fastai import *
from fastai.vision.all import *
from IPython.display import clear_output, DisplayHandle
import cv2 

print(torch.cuda.is_available())

def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)
DisplayHandle.update = update_patch
# clear the GPU cache
torch.cuda.empty_cache()

batch_size = 16


True


In [7]:
# load up the data
def get_all_paths(directory_path):
    directory = Path(directory_path)
    paths = []
    for path in directory.glob('**/*'):
        if path.is_file():
            paths.append(path)
    return paths


project_directory = Path.cwd().joinpath('../')
train_mask_path = project_directory.joinpath('data/Lauren_PreEclampsia_Data/train/masks')
glom_mask_files = get_all_paths(train_mask_path)
print(len(glom_mask_files))

train_image_path = project_directory.joinpath('data/Lauren_PreEclampsia_Data/train/images')
glom_image_files = get_all_paths(train_image_path)
print(len(glom_image_files))


88
84


In [8]:
# new addition is thresh, since for whatever reason my binary mask has pixel values in between 0 and 1
def n_glom_codes(fnames, is_partial=True):
  "Gather the codes from a list of `fnames`, full file paths"
  vals = set()
  if is_partial:
    random.shuffle(fnames)
    fnames = fnames[:10]
  for fname in fnames:
    msk = np.array(PILMask.create(fname))
    for val in np.unique(msk):
      if val not in vals:
        vals.add(val)
  vals = list(vals)
  p2c = dict()
  for i, val in enumerate(vals):
    p2c[i] = vals[i]
  return p2c


p2c = n_glom_codes(glom_mask_files)
print(p2c)


def get_glom_mask_file(image_file, p2c, thresh=127):
    # this is the base path
    base_path = image_file.parent.parent.parent
    # print(base_path)
    first_name = image_file.parent.name
    # print(first_name)
    # get training or testing from here
    full_name = re.findall(string=image_file.name, pattern=r"^[A-Za-z]*[0-9]+[_|-]+[A-Za-z]*[0-9]+")[0]
    # put the whole thing together
    str_name = f'{full_name}_mask' + image_file.suffix
    # attach it to the correct path
    mask_path = (base_path / 'masks' / first_name / str_name)

    # convert to an array (mask)
    msk = np.array(PILMask.create(mask_path))
    # convert the image to binary if it isn't already (tends to happen when working with .jpg files)
    msk[msk <= thresh] = 0
    msk[msk > thresh] = 1

    # find all the possible values in the mask (0,255)
    for i, val in enumerate(p2c):
        msk[msk == p2c[i]] = val
    return PILMask.create(msk)


def get_glom_y(o):
    return get_glom_mask_file(o, p2c)


{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 128, 10: 129, 11: 130, 12: 131, 13: 132, 14: 133, 15: 134, 16: 255, 17: 136, 18: 57, 19: 58, 20: 135, 21: 187, 22: 188, 23: 189, 24: 184, 25: 185, 26: 186, 27: 59, 28: 60, 29: 61, 30: 62, 31: 63, 32: 64, 33: 65, 34: 66, 35: 67, 36: 68, 37: 69, 38: 70, 39: 190, 40: 191, 41: 192, 42: 193, 43: 194, 44: 195, 45: 196, 46: 197, 47: 199, 48: 71, 49: 247, 50: 120, 51: 253, 52: 249, 53: 198, 54: 250, 55: 251, 56: 252, 57: 246, 58: 254, 59: 248, 60: 121, 61: 122, 62: 123, 63: 124, 64: 125, 65: 126, 66: 127}


In [23]:
path_to_learner = project_directory.joinpath('segmentation_model_dir/glomerulus_segmentation_model-dynamic_unet-e50_b16_s84.pkl')
glom_segmentation_learner = load_learner(path_to_learner)


In [10]:
def get_roi(f):
    image = PILImage.create(f)
    mask = get_glom_mask_file(f, p2c)
    roi = Image.fromarray(np.array(image) * np.expand_dims(np.array(mask), -1))
    return roi

def make_roi_dirs(image_files, train_test = 'train'):
    path_to_rois = project_directory.joinpath(f'data/Lauren_PreEclampsia_Data/{train_test}/rois')
    # Create a directory called "rois"
    os.makedirs(path_to_rois, exist_ok = True)

    # Get the ROIs for each image and save them to the "rois" directory
    for i, f in enumerate(image_files):
        roi = get_roi(f)
        filename = os.path.basename(f)
        dir_name = Path(f).parent.parts[-1]
        dir_ = f'{path_to_rois}/{dir_name}/'
        os.makedirs(dir_, exist_ok=True)
        full_file_path = f"{dir_}/{filename}"
        # print(full_file_path)
        roi.save(full_file_path)



In [11]:
make_roi_dirs(glom_image_files, train_test='train')

In [12]:
roi_path = project_directory.joinpath('data/Lauren_PreEclampsia_Data/train/rois')
glom_roi_files = get_all_paths(roi_path)


In [13]:
gpt_rec_batch_aug = [*aug_transforms(size=256,  # Increase the output image size
                                     flip_vert=True,
                                     max_rotate=45,
                                     min_zoom=0.8,
                                     max_zoom=1.3,
                                     max_warp=0.4,
                                     max_lighting=0.2),  # Add brightness and contrast adjustments
                     RandomErasing(p=0.5, sl=0.01, sh=0.3, min_aspect=0.3, max_count=3)]

rois = DataBlock(blocks=(ImageBlock),
                  splitter=RandomSplitter(valid_pct=0.2, seed=42),
                  get_items=lambda x: glom_roi_files,
                  item_tfms=[RandomResizedCrop(512, min_scale=0.45)],  # this is super important - upscale the crop at each batch randomly
                  batch_tfms=gpt_rec_batch_aug,
                  n_inp=1,
                  )

batch_size = 16
roi_dls = rois.dataloaders(glom_roi_files,  bs=batch_size)
rois.summary(glom_roi_files, bs=batch_size)


Setting-up type transforms pipelines
Found 84 items
2 datasets of sizes 68,16
Setting up Pipeline: PILBase.create

Building one sample
  Pipeline: PILBase.create
    starting from
      /home/ncamarda/coding/endotheliosisQuantifier/scripts/../data/Lauren_PreEclampsia_Data/train/rois/T29/T29_Image0.jpg
    applying PILBase.create gives
      PILImage mode=RGB size=2448x2048

Final sample: (PILImage mode=RGB size=2448x2048,)


Found 84 items
2 datasets of sizes 68,16
Setting up Pipeline: PILBase.create
Setting up after_item: Pipeline: RandomResizedCrop -- {'size': (512, 512), 'min_scale': 0.45, 'ratio': (0.75, 1.3333333333333333), 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'val_xtra': 0.14, 'max_scale': 1.0, 'p': 1.0} -> ToTensor
Setting up before_batch: Pipeline: 
Setting up after_batch: Pipeline: IntToFloatTensor -- {'div': 255.0, 'div_mask': 1} -> Dihedral -- {'size': 256, 'mode': 'bilinear', 'pad_mode': 'reflection', 'mode_mask': 'nearest', 'align_corners': Tru

In [16]:
print(glom_segmentation_learner.model)


DynamicUnet(
  (layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05

In [24]:
class SaveFeatures:
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)

    def hook_fn(self, module, input, output):
        self.features = output

    def close(self):
        self.hook.remove()

# Extract the pytorch model from the fastai learner module
glom_seg_model = glom_segmentation_learner.model
glom_seg_model = glom_seg_model.cuda()  # Move the model to GPU

# Assuming the last layer before upsampling and segmentation is layer number 6
layer_to_hook = glom_seg_model.layers[6]
saved_features = SaveFeatures(layer_to_hook)


In [25]:
# Get a batch from the dataloader
batch = next(iter(roi_dls.train))
images = batch[0]

# Register the hook
saved_features = SaveFeatures(glom_seg_model[-3])

# Perform the forward pass with the model
with torch.no_grad():
    _ = glom_seg_model(images)

# Access the features
features = saved_features.features
print(features)


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [18]:
roi_features = []

for batch_idx, batch in enumerate(roi_dls.train):
    images = batch
    with torch.no_grad():
        _ = (images)
    features = saved_features.features
    # features is a tensor of shape (batch_size, C, H, W), where C=64
    roi_features.extend(features.cpu().numpy())

saved_features.close()  # Remove the hook


TypeError: hasattr(): attribute name must be string

In [ ]:
import matplotlib.pyplot as plt


def show_feature_map(features, roi_index, channel_index):
    # Extract the specific feature map for a given ROI and channel
    feature_map = features[roi_index, channel_index]

    # Normalize the feature map to [0, 1]
    feature_map = (feature_map - feature_map.min()) / (feature_map.max() - feature_map.min())

    # Display the feature map as an image
    plt.imshow(feature_map, cmap='gray')
    plt.show()


# Example usage
roi_index = 0  # Index of the ROI you want to visualize
channel_index = 0  # Index of the channel you want to visualize
show_feature_map(all_features, roi_index, channel_index)
